<a href="https://colab.research.google.com/github/asigalov61/SuperPiano/blob/master/Super_Piano_3_Alt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Super Piano 3
## Google Music Transformer

### All thanks and credit for this colab go out to Prayag Chatha on whose repo and code it is based: https://github.com/chathasphere/pno-ai

In [ ]:
#@title Clone pno-ai repo and install dependencies
!git clone 'https://github.com/asigalov61/pno-ai'
!pip install pretty_midi
!pip install rtmidi

In [ ]:
#@title Import modules and setup main (hyper)parameters
%cd /content/pno-ai/
import os, time, datetime
import torch
import torch.nn as nn
torch.set_default_tensor_type(torch.cuda.FloatTensor)
from random import shuffle
from preprocess import PreprocessingPipeline
from train import train
from model import MusicTransformer

n_epochs = 60
batch_size = 128
sampling_rate = 125
n_velocity_bins = 32
seq_length = 256
n_tokens = 256 + sampling_rate + n_velocity_bins
transformer = MusicTransformer(n_tokens, seq_length, 
        d_model = 128, n_heads = 8, d_feedforward=512, 
        depth = 6, positional_encoding=True, relative_pos=True)

In [ ]:
#@title Download Google Magenta MAESTRO v.2.0.0 Piano MIDI Dataset (~1300 MIDIs)
%cd /content/pno-ai/data
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip -j maestro-v2.0.0-midi.zip
!rm maestro-v2.0.0-midi.zip
%cd /content/pno-ai

In [ ]:
%cd /content/pno-ai/data
!unzip -j '/content/pno-ai/data/Super-Piano-2-Performance-DataSet-CC-BY-NC-SA.zip'
!rm '/content/pno-ai/data/Super-Piano-2-Performance-DataSet-CC-BY-NC-SA.zip'

In [ ]:
#@title Process custom MIDI dataset
%cd /content/pno-ai
pipeline = PreprocessingPipeline(input_dir="data", stretch_factors=[0.975, 1, 1.025],
           split_size=30, sampling_rate=sampling_rate, n_velocity_bins=n_velocity_bins,
           transpositions=range(-2,3), training_val_split=0.9, max_encoded_length=seq_length+1,
           min_encoded_length=257)

pipeline_start = time.time()
pipeline.run()
runtime = time.time() - pipeline_start
print(f"MIDI pipeline runtime: {runtime / 60 : .1f}m")

In [ ]:
#@title Train the model
%cd '/content/pno-ai'
today = datetime.date.today().strftime('%m%d%Y')
checkpoint = f"saved_models/tf_{today}"

training_sequences = pipeline.encoded_sequences['training']
validation_sequences = pipeline.encoded_sequences['validation']


train(transformer, training_sequences, validation_sequences,
               epochs = n_epochs, evaluate_per = 1, custom_schedule=True, custom_loss=True,
               batch_size = batch_size, batches_per_print=100,
               padding_index=0, checkpoint_path=checkpoint)

In [ ]:
#@title Save the model
%cd /content/pno-ai
torch.save(transformer, 'trained_model.pth')

In [ ]:
#@title Load pre-trained checkpoint if needed
%cd /content/pno-ai
transformer.load_state_dict(torch.load('/content/pno-ai/saved_models/tf_09232020_e4'))
print("Successfully loaded checkpoint!")
transformer.eval()

In [ ]:
#@title Define Helper Functions
import numpy as np
from pretty_midi import Note, PrettyMIDI, Instrument
import torch.nn.functional as F
import copy, pathlib

def vectorize(sequence):
    """
    Converts a list of pretty_midi Note objects into a numpy array of
    dimension (n_notes x 4)
    """
    array = [[note.start, note.end, note.pitch, note.velocity] for
            note in sequence]
    return np.asarray(array)

def devectorize(note_array):
    """
    Converts a vectorized note sequence into a list of pretty_midi Note
    objects
    """
    return [Note(start = a[0], end = a[1], pitch=a[2],
        velocity=a[3]) for a in note_array.tolist()]


def one_hot(sequence, n_states):
    """
    Given a list of integers and the maximal number of unique values found
    in the list, return a one-hot encoded tensor of shape (m, n)
    where m is sequence length and n is n_states.
    """
    if torch.cuda.is_available():
        return torch.eye(n_states)[sequence,:].cuda()
    else:
        return torch.eye(n_states)[sequence,:]

def decode_one_hot(vector):
    '''
    Given a one-hot encoded vector, return the non-zero index
    '''
    return vector.nonzero().item()

def prepare_batches(sequences, batch_size):
    """
    Splits a list of sequences into batches of a fixed size. Each sequence yields an input sequence
    and a target sequence, with the latter one time step ahead. For example, the sequence "to be or not
    to be" gives an input sequence of "to be or not to b" and a target sequence of "o be or not to be."
    """
    n_sequences = len(sequences)
    for i in range(0, n_sequences, batch_size):
        batch = sequences[i:i+batch_size]
	#needs to be in sorted order for packing batches to work
        batch = sorted(batch, key = len, reverse=True)
        input_sequences, target_sequences = [], []

        for sequence in batch:
            input_sequences.append(sequence[:-1])
            target_sequences.append(sequence[1:])

        yield input_sequences, target_sequences

def clones(module, N):
    "Clone N identical layers of a module"
    return torch.nn.ModuleList([copy.deepcopy(module) for i in range(N)])

def d(tensor=None):
    if tensor is None:
        return 'cuda' if torch.cuda.is_available() else 'cpu'
    return 'cuda' if tensor.is_cuda else 'cpu'

def write_midi(note_sequence, output_dir, filename):

    #make output directory
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    #generate midi
    midi = PrettyMIDI()
    piano_track = Instrument(program=0, is_drum=False, name=filename)
    piano_track.notes = note_sequence
    midi.instruments.append(piano_track)
    output_name = output_dir + f"{filename}.midi"
    midi.write(output_name)

def sample(model, sample_length, prime_sequence=[], temperature=1):
    """
    Generate a MIDI event sequence of a fixed length by randomly sampling from a model's distribution of sequences. Optionally, "seed" the sequence with a prime. A well-trained model will create music that responds to the prime and develops upon it.
    """
    #deactivate training mode
    model.eval()
    if len(prime_sequence) == 0:
        #if no prime is provided, randomly select a starting event
        input_sequence = [np.random.randint(model.n_tokens)]
    else:
        input_sequence = prime_sequence.copy()

    #add singleton dimension for the batch
    input_tensor = torch.LongTensor(input_sequence).unsqueeze(0).cuda()

    for i in range(sample_length):
        #select probabilities of *next* token
        out = model(input_tensor)[0, -1, :]
        #out is a 1d tensor of shape (n_tokens)
        probs = F.softmax(out / temperature, dim=0)
        #sample prob distribution for next character
        c = torch.multinomial(probs,1)
        input_tensor = torch.cat([input_tensor[:,1:], c[None]], dim=1)
        input_sequence.append(c.item())

    return input_sequence

In [ ]:
#@title Generate Output
import uuid
import torch
from torch import nn
torch.set_default_tensor_type(torch.cuda.FloatTensor)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
from model import MusicTransformer
from preprocess import SequenceEncoder
import midi_input


class GeneratorError(Exception):
    pass

model_key = ''

    
n_velocity_events = 32
n_time_shift_events = 125

decoder = SequenceEncoder(n_time_shift_events, n_velocity_events,
           min_events=0)

prime_sequence = []



temp = 0.4

trial_key = str(uuid.uuid4())[:6]
n_trials = 1

sample_l = 2048

keep_g = False
stuck_note_d = None
note_sequence = []


for i in range(n_trials):
            print("generating sequence")
            output_sequence = sample(transformer, prime_sequence = prime_sequence, sample_length=sample_l, temperature=temp)
            note_sequence = decoder.decode_sequence(output_sequence, 
                verbose=True, stuck_note_duration=stuck_note_d, keep_ghosts=keep_g)

            output_dir = "/content/pno-ai"
            write_midi(note_sequence, output_dir, 'output')